# Data Report -
### Energy consumption as driver of greenhouse gas emissions and the influence of renewables.

## Main Question
The main question of this project will discuss is:

1. How does the amount of energy consumed influence the net greenhouse gas emissions of European countries?

Furthermore, the project will answer the follow-up question:

2. And how is this influenced by the share of renewables in total energy?

## Data Sources
This project utilizes three data sources from the European data portal [Eurostat](https://ec.europa.eu/eurostat/). Eurostat was chosen for this project because it provides high-quality, comparable data for all EU countries, not just individual nations. This consistency is crucial for a comprehensive analysis.

Eurostat's supra-national approach to data collection and publication is particularly advantageous for this project, as tackling climate change necessitates a broad, as-global-as-possible perspective. By examining data across 27 EU countries, the main question of 'how energy consumption influences greenhouse gas emissions' can be addressed in a more generalized manner.

Moreover, the significant variations in renewable energy usage across these countries allow for a robust analysis of the secondary question: 'how the correlation between energy consumption and greenhouse gas emissions is influenced by the share of renewables'. This comparative analysis across 27 countries enhances the generalizability of the findings.

**Structure of the data sources:** The data sources are similarly well-structured in a tabular form and explanations for which column has what meaning is provided via each source's metadata.

---
#### Datasource 1: Net greenhouse gas emissions
Describes the net greenhouse gas emissions for all countries in the EU either relative to 1990 or as tonnes per capita.

**Data Quality:** The data appears to be of high quality, with no missing cells, consistent formatting, and relevance to this project. However, there are some limitations: the accuracy cannot be easily verified, and some data cells are flagged as estimates or provisional.

---
#### Datasource 2: Primary energy consumption
Describes energy consumption of end users like households or industry and the energy consumption of the energy sector itself, accounting for energy transformation costs. The data is available for all countries of the EU. Data is measured either as million tonnes of oil equivalent, relative to 2005 or as tonnes of oil equivalent per capita.

**Data Quality:** Data is missing for multiple countries and years, lowering the completeness of the dataset. For example, Switzerland has no recorded data on primary energy consumption, and several countries lack data for the year 2022. On the positive side, the formatting of the data is consistent, and the source is relevant to this project. However, as with Datasource 1, there are some limitations: the accuracy cannot be easily verified. 

---
#### Datasource 3: Share of energy from renewable sources
The share of energy generated from renewable sources as declared by the EU. The data is available for all countries of the EU and is measured in percentages.

**Data Quality:** The data appears to be of great quality, it is consistently formatted, has high relevance to the second question and is almost complete, only a few cells are missing for the countries Bosnia-and-Herzegovina and Georgia. As with Datasource 1 and 2, there are some limitations: the accuracy cannot be easily verified. 

---

The age of all data sources is not a concern, as the measurements span multiple years and represent fixed snapshots that will not change over time.

### Licenses
All data sources on Eurostat are subject to the same [copyright notice](https://ec.europa.eu/eurostat/about-us/policies/copyright), which states as follows:
> Eurostat has a policy of encouraging free re-use of its data, both for non-commercial and commercial purposes. All statistical data, metadata, content of web pages or other dissemination tools, official publications and other documents published on its website, with the exceptions listed below, can be reused without any payment or written licence provided that:
> - the source is indicated as Eurostat;
> - when re-use involves modifications to the data or text, this must be stated clearly to the end user of the information.

This grants the project the right to use any of Eurostat's data sources. In order to adhere to the obligations outlined in the copyright notice, the project will attribute Eurostat as the data's source and denote any modifications made to the data.

## Data Pipeline

## Results and Limitations